In [2]:
# !pip install pytesseract
# !pip install tesseract
# !pip install google-cloud-vision

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

grpcio-status 1.46.0 requires grpcio>=1.46.0, but you'll have grpcio 1.44.0 which is incompatible.
You should consider upgrading via the 'c:\users\ppetropo\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import pytesseract
import numpy as np
# import os
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='auth.json'

# from google.cloud import vision

# # re is used to extract IMEI using regex
# import re

In [17]:
# read image
img = cv2.imread('handwritten1.jpg')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
# cv2.imshow(img)
config = r'--oem 3 --psm 6'



# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 


In [20]:
gray = get_grayscale(img)
thresh = thresholding(gray)
opening = opening(gray)
canny = canny(gray)

In [18]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Users/ppetropo/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'

text = pytesseract.image_to_string(img, config=config)
# print text
text = text.split('\n')
print(text)
text = ' '.join(text)
print(text)

['L ive in Luv.', 'Fvery day [ go', 'to work ly Lis', 'Also I would', 'like £0 visit Hars', '']
L ive in Luv. Fvery day [ go to work ly Lis Also I would like £0 visit Hars 


In [19]:
import nltk
import re

In [20]:
t = nltk.tokenize.word_tokenize(text)
print(t)
nltk.pos_tag(t)

['L', 'ive', 'in', 'Luv', '.', 'Fvery', 'day', '[', 'go', 'to', 'work', 'ly', 'Lis', 'Also', 'I', 'would', 'like', '£0', 'visit', 'Hars']


[('L', 'NNP'),
 ('ive', 'NN'),
 ('in', 'IN'),
 ('Luv', 'NNP'),
 ('.', '.'),
 ('Fvery', 'NNP'),
 ('day', 'NN'),
 ('[', 'NNP'),
 ('go', 'VBP'),
 ('to', 'TO'),
 ('work', 'VB'),
 ('ly', 'NN'),
 ('Lis', 'NNP'),
 ('Also', 'RB'),
 ('I', 'PRP'),
 ('would', 'MD'),
 ('like', 'VB'),
 ('£0', 'NNP'),
 ('visit', 'NN'),
 ('Hars', 'NNP')]

In [21]:

import spacy

# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])



# create spacy 
doc = nlp(text)

for token in doc:
    print(token.text,'->',token.pos_)

L -> NOUN
i -> PRON
ve -> VERB
in -> ADP
Luv -> PROPN
. -> PUNCT
Fvery -> NOUN
day -> NOUN
[ -> PUNCT
go -> VERB
to -> PART
work -> VERB
ly -> ADP
Lis -> PROPN
Also -> ADV
I -> PRON
would -> AUX
like -> VERB
£ -> SYM
0 -> NUM
visit -> NOUN
Hars -> NOUN


In [22]:
for token in doc:
    # check token pos
    if token.pos_=='NOUN':
        # print token
        print(token.text)

L
Fvery
day
visit
Hars


In [15]:
# create spacy 
doc = nlp(text)

for token in doc:
    print(token.text,'->',token.pos_)

HOW -> SCONJ
TO -> PART
WRITE -> VERB
ALT -> PROPN
TEXT -> PROPN
AND -> CCONJ
IMAGE -> NOUN
DESCRIPTIONS -> NOUN
FOR -> ADP
} -> PUNCT
INSTAGRAM -> NOUN


In [16]:
from spacy import displacy 
displacy.render(doc, style='dep',jupyter=True)